# Pydantic Configurations

> - Documentation: https://pydantic-docs.helpmanual.io/
> - Configuration for Pydantic models: https://docs.pydantic.dev/latest/api/config/



```python
from pydantic import BaseModel, ConfigDict

class Model(BaseModel):
    model_config = ConfigDict()

```

## Handling Extra Fields

In [223]:
from pydantic import BaseModel, ConfigDict, ValidationError

class Model(BaseModel):
    field_1: int

In [224]:
display(Model.model_config)

{}

In [225]:
# This will not raise an error because extra fields are allowed and ignored by default
m1 = Model(field_1=1, field_2=2)
print(m1)
print(m1.model_fields)
print(m1.model_fields_set)

# See extra fields
print(m1.model_extra)

field_1=1
{'field_1': FieldInfo(annotation=int, required=True)}
{'field_1'}
None


__extra__: 

Whether to ignore, allow, or forbid extra attributes during model initialization. Defaults to 'ignore'.
You can configure how pydantic handles the attributes that are not defined in the model:

- `allow` - Allow any extra attributes.
- `forbid` - Forbid any extra attributes.
- `ignore` - Ignore any extra attributes.


Doc: https://docs.pydantic.dev/latest/api/config/#pydantic.config.ConfigDict.extra

In [226]:
class Model(BaseModel):
    model_config = ConfigDict(extra="forbid")

    field_1: int

In [227]:
display(Model.model_config)

{'extra': 'forbid'}

In [228]:
# This will raise an error because extra fields are forbidden
try:
    m2 = Model(field_1=1, field_2=2)
except ValidationError as e:
    print(e)

1 validation error for Model
field_2
  Extra inputs are not permitted [type=extra_forbidden, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/extra_forbidden


In [229]:
class Model(BaseModel):
    model_config = ConfigDict(extra="allow")

    field_1: int

In [230]:
display(Model.model_config)

{'extra': 'allow'}

In [231]:
# This will not raise an error because extra fields are allowed
m3 = Model(field_1=1, field_2=2)
display(m3)
print("model fields",m3.model_fields)
print("fields set", m3.model_fields_set)

# See extra fields
print("extra fields",m3.model_extra)

Model(field_1=1, field_2=2)

model fields {'field_1': FieldInfo(annotation=int, required=True)}
fields set {'field_1', 'field_2'}
extra fields {'field_2': 2}


## Strict and Lax Mode (Type Coercion)

In [232]:
class Model(BaseModel):
    field_1: str
    field_2: float
    field_3: list
    field_4: tuple

In [233]:
try:
    Model(field_1=9999, field_2=2, field_3=(1,2,3), field_4=[1,2,3])
except ValidationError as e:
    print(e)

1 validation error for Model
field_1
  Input should be a valid string [type=string_type, input_value=9999, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type


In [234]:
Model(field_1="9999", field_2=2, field_3=(1,2,3), field_4=[1,2,3])

Model(field_1='9999', field_2=2.0, field_3=[1, 2, 3], field_4=(1, 2, 3))

In [235]:
# Set strict mode to True
class Model(BaseModel):
    model_config = ConfigDict(strict=True)

    field_1: str
    field_2: float
    field_3: list
    field_4: tuple

In [236]:
try:
    Model(field_1="9999", field_2=2, field_3=(1,2,3), field_4=[1,2,3])
except ValidationError as e:
    print(e)

2 validation errors for Model
field_3
  Input should be a valid list [type=list_type, input_value=(1, 2, 3), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.8/v/list_type
field_4
  Input should be a valid tuple [type=tuple_type, input_value=[1, 2, 3], input_type=list]
    For further information visit https://errors.pydantic.dev/2.8/v/tuple_type


https://docs.pydantic.dev/latest/concepts/conversion_table/#__tabbed_1_2

In [237]:
json_data = """
{
    "field_1": true,
    "field_2": 10,
    "field_3": 1,
    "field_4": null,
    "field_5": [1, 2, 3],
    "field_6": ["a", "b", "c"],
    "field_7": {"a": 1, "b": 2}
}
"""

In [238]:
class Model(BaseModel):
    model_config = ConfigDict(strict=True)

    field_1: bool
    field_2: float
    field_3: int
    field_4: str | None
    field_5: tuple[int, ...]
    field_6: set[str]
    field_7: dict

In [239]:
data = Model.model_validate_json(json_data)
display(data)

Model(field_1=True, field_2=10.0, field_3=1, field_4=None, field_5=(1, 2, 3), field_6={'a', 'c', 'b'}, field_7={'a': 1, 'b': 2})

In [240]:
json_data = """
{
    "field_1": true,
    "field_2": 10,
    "field_3": 1,
    "field_4": null,
    "field_5": [1, 2, 3.5],
    "field_6": ["a", "b", 100],
    "field_7": {"a": 1, "b": 2}
}
"""

try:
    Model.model_validate_json(json_data)
except ValidationError as ex:
    print(ex)

2 validation errors for Model
field_5.2
  Input should be a valid integer [type=int_type, input_value=3.5, input_type=float]
    For further information visit https://errors.pydantic.dev/2.8/v/int_type
field_6.2
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type
